In [1]:
from IPython.display import Javascript
from plotly.offline import get_plotlyjs
Javascript(get_plotlyjs())

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, date

from dash import Dash, dcc, html, Input, Output
from jupyter_dash import JupyterDash
import plotly.express as px

#import plotly.io as pio
#pio.renderers.default = "notebook"

# Apresentação Alforriah
## Bruno Perdigão

#### Apresentação

- Me chamo Bruno Perdigão. Sou arquiteto e trabalho com desenvolvimento de projetos de edificações.
- Comecei a estudar python para desenvolver meus conhecimentos sobre design computacional e acabei entrando em contato com o mundo da Ciência de Dados.
- Atualmente estou me qualificando nesta área e busco aliar meus novos conhecimentos com a minha experiênciam em arquitetura.

![bg 75%](Python-Energy+.png)

![bg 80%](curso-edx.png)
Curso criado por Clayton Miller - Professor assistente na National University of Singapore

### O conjunto de dados

Os dados utilizados para esse trabalho foram fornecidos no curso do prof. Clayton Miller mencionado anteriormente. É formado pela mediação de gastos energéticos de edificações, medido por sensores conectados aos edifícios. Parte desse conjunto é formado por dados publicamente disponíveis e outros foram feitos a partir da coleta privada em reuniões e visitas à setores de manutenção de edifícios.

![bg 32%](dataframe-dados-eletricos.png)


Para manter a privacidade dos dados não publicizados foram utilizados "apelidos" para nomear o local e o edifício. A nomenclatura das colunas obedece a seguinte lógica: (Apelido do Local)_(Uso da edificação)_(Apelido do Edifício). Todas as medições dos dados de consumo elétricos foram convertidos para __kWh__.

In [2]:
df = pd.read_csv("D:/Sync/Profissional/Análise de Dados/Conjunto de Dados/building-data-genome-project-2/data/weather/weather.csv", index_col='timestamp', parse_dates=True)
df = df.iloc[:,:2]
df = df.fillna(method='ffill')
df

,site_id,airTemperature
timestamp,,
2016-01-01 00:00:00,Panther,19.4
2016-01-01 01:00:00,Panther,21.1
2016-01-01 02:00:00,Panther,21.1
2016-01-01 03:00:00,Panther,20.6
2016-01-01 04:00:00,Panther,21.1
...,...,...
2017-12-31 19:00:00,Mouse,8.5
2017-12-31 20:00:00,Mouse,8.5
2017-12-31 21:00:00,Mouse,8.2


In [3]:
df = df.pivot(columns='site_id', values='airTemperature')
df = df.resample("D").mean() # Só funciona com datetime como index
df = df.reset_index()
df = df.melt(id_vars=['timestamp'], value_vars=df.columns)
df = df.rename(columns={'value': 'airTemperature'})
df



,timestamp,site_id,airTemperature
0,2016-01-01,Bear,6.175000
1,2016-01-02,Bear,8.087500
2,2016-01-03,Bear,10.112500
3,2016-01-04,Bear,11.270833
4,2016-01-05,Bear,10.962500
...,...,...,...
13884,2017-12-27,Wolf,0.862500
13885,2017-12-28,Wolf,0.762500
13886,2017-12-29,Wolf,4.787500
13887,2017-12-30,Wolf,6.687500


In [4]:
'''
app = JupyterDash(__name__)
app.layout = html.Div([
    html.Div([
        dcc.Markdown('# Título')
    ]),
    html.Div([
        dcc.Dropdown(df['site_id'].unique(), 'Panther', id='site_id_drop')
    ]),
    html.Div([
        dcc.Graph(id='temperature_by_site')
    ])
])

@app.callback(
    Output('temperature_by_site', 'figure'),
    Input('site_id_drop', 'value')
)
def update_graph(dropdown_value):    
    fig = px.line(df, x=df[df['site_id'] == dropdown_value]['timestamp'], y=df[df['site_id'] == dropdown_value]['airTemperature'])

    return fig

app.run_server(mode='inline')'''

"\napp = JupyterDash(__name__)\napp.layout = html.Div([\n    html.Div([\n        dcc.Markdown('# Título')\n    ]),\n    html.Div([\n        dcc.Dropdown(df['site_id'].unique(), 'Panther', id='site_id_drop')\n    ]),\n    html.Div([\n        dcc.Graph(id='temperature_by_site')\n    ])\n])\n\n@app.callback(\n    Output('temperature_by_site', 'figure'),\n    Input('site_id_drop', 'value')\n)\ndef update_graph(dropdown_value):    \n    fig = px.line(df, x=df[df['site_id'] == dropdown_value]['timestamp'], y=df[df['site_id'] == dropdown_value]['airTemperature'])\n\n    return fig\n\napp.run_server(mode='inline')"

In [5]:
start = df['timestamp'].min().to_pydatetime()
end = df['timestamp'].max().to_pydatetime()
start, end

(datetime.datetime(2016, 1, 1, 0, 0), datetime.datetime(2017, 12, 31, 0, 0))

In [7]:
app = JupyterDash(__name__)
app.layout = html.Div([
    html.Div([
        dcc.Markdown('# Dados de temperatura de cada local')
    ]),
    html.Div([
        dcc.Checklist(
            df['site_id'].unique(),
            ['Panther'],
            id='site_id_drop')
    ]),
    html.Div([
        dcc.Graph(id='temperature_by_site')
    ]),
    html.Div([
        dcc.DatePickerRange(
            start_date=start,
            end_date=end,
            min_date_allowed=start,
            max_date_allowed=end,
            initial_visible_month=start,
            calendar_orientation='vertical',
            display_format='DD/MM/YYYY',
            id='date_picker')   
    ])
])

@app.callback(
    Output('temperature_by_site', 'figure'),
    Input('site_id_drop', 'value'),
    Input('date_picker', 'start_date'),
    Input('date_picker', 'end_date')
)
def update_graph(dropdown_value, start, end):
    idf = df[(df['timestamp'] > start) & (df['timestamp'] < end)]

    fig = px.line(idf,
                x=idf[idf['site_id'].isin(dropdown_value)]['timestamp'],
                y=idf[idf['site_id'].isin(dropdown_value)]['airTemperature'],
                color=idf[idf['site_id'].isin(dropdown_value)]['site_id']
        )

    return fig

app.run_server(mode='inline')